In [1]:
import bw2io as bi
import bw2data as bd
from pathlib import Path
import randonneur as rn
import randonneur_data as rd

In [2]:
bd.projects.set_current("agrobalyse-3.1.1 randonneur")

In [3]:
if "ecoinvent-3.8-cutoff" not in bd.databases:
    bi.ecoinvent.import_ecoinvent_release("3.8", "cutoff", lcia=False)

In [4]:
if "ecoinvent-3.10-cutoff" not in bd.databases:
    bi.ecoinvent.import_ecoinvent_release("3.10", "cutoff")

In [3]:
fp = Path("/Users/cmutel/Projects/Agribalyse 3.1 import/3.1.1/AGB 3.1.1.CSV")

In [4]:
sp = bi.SimaProBlockCSVImporter(fp, "agribalyse-3.1.1", separate_products=False)

2024-11-26 15:30:33.681 | INFO     | bw_simapro_csv.main:__init__:119 - Writing logs to /Users/cmutel/Library/Logs/bw_simapro_csv/AGB 3.1.1-2024-11-26T15-30-33
2024-11-26 15:30:33.692 | INFO     | bw_simapro_csv.main:__init__:142 - Using database name 'agribalyse-3.1.1'
2024-11-26 15:30:33.693 | INFO     | bw_simapro_csv.main:__init__:147 - SimaPro CSV import started.
	File: '/Users/cmutel/Projects/Agribalyse 3.1 import/3.1.1/AGB 3.1.1.CSV'
	Delimiter: ';'
	Name: 'AGRIBALYSE - Unit'
2024-11-26 15:31:48.460 | INFO     | bw_simapro_csv.main:resolve_parameters:316 - Extracted and cleaned 17557 process datasets


In [5]:
sp.statistics()

Graph statistics for `agribalyse-3.1.1` importer:
19044 graph nodes:
	process: 17070
	readonly_process: 1487
	multifunctional: 487
5076133 graph edges:
	biosphere: 4911160
	technosphere: 124368
	production: 40605
2974 edges to the following databases:
	agribalyse-3.1.1: 2974
33716 unique unlinked edges (5073159 total):
	technosphere: 14458
	production: 17069
	biosphere: 2189




(19044, 5076133, 5073159, 487)

In [18]:
[x['name'] for x in sp.data if 'coinvent' in x['name']][:200]

['Ammonium nitrate phosphate, as N, at plant (WFLDB 3.5 updated with Ecoinvent 3.8',
 'Ammonium nitrate phosphate, as N, at plant (WFLDB 3.5 updated with Ecoinvent 3.8',
 'Ammonium nitrate phosphate, as N, at plant (WFLDB 3.5 updated with Ecoinvent 3.8',
 'Ammonium nitrate phosphate, as P2O5, at plant (WFLDB 3.5 updated with Ecoinvent',
 'Ammonium nitrate phosphate, as P2O5, at plant (WFLDB 3.5 updated with Ecoinvent',
 'Ammonium nitrate phosphate, as P2O5, at plant (WFLDB 3.5 updated with Ecoinvent',
 'Calcium ammonium nitrate (27% N), at plant (WFLDB 3.5 updated with Ecoinvent 3.8',
 'Calcium ammonium nitrate (27% N), at plant (WFLDB 3.5 updated with Ecoinvent 3.8',
 'Calcium ammonium nitrate (27% N), at plant (WFLDB 3.5 updated with Ecoinvent 3.8',
 'Grazed grass, at farm, country mix, per kg DM (WFLDB 3.5 updated with Ecoinvent',
 'Tomato, processing grade, open field, at farm (WFLDB 3.5 updated with Ecoinvent',
 'Tomato, processing grade, open field, at farm (WFLDB 3.5 updated wit

In [19]:
def single_functional_input(ds: dict) -> dict:
    fe = [exc for exc in ds['exchanges'] if exc.get("functional") and exc.get("type") == "technosphere"]
    if len(fe) == 1:
        return fe[0]
    return {}

In [34]:
data = {
    (ds['code'], ds['name'])
    for ds in sp.data 
    if ds['tags'].get('system_description') == 'Ecoinvent v3 - Copied from Ecoinvent'
    and any(
        (
            exc['name'].endswith("S - Copied from Ecoinvent U") 
            or exc['name'].endswith("Cut-off, S - Copied from Ecoinvent {GLO} U") 
        )
        for exc in ds['exchanges'] 
        if exc.get("functional")
    )
}.union({
    (ds['code'], ds['name'])
    for ds in sp.data 
    if 'Cut-off, S - Copied from Ecoinvent' in single_functional_input(ds).get("name", "")
    and "(waste treatment)" in single_functional_input(ds).get("name", "")
})

In [36]:
[x for x in data if x[0] == 'AGRIBALU000000003110690']

[('AGRIBALU000000003110690', 'market for p-dichlorobenzene RER')]

In [28]:
ds = [x for x in sp.data if x['code'] == 'AGRIBALU000000003110690'][0]

In [29]:
ds['tags'].get('system_description') == 'Ecoinvent v3 - Copied from Ecoinvent'

True

In [31]:
[
    exc['name']
    for exc in ds['exchanges'] 
    if exc.get("functional")
]

['P-dichlorobenzene {RER}| market for p-dichlorobenzene | Cut-off, S - Copied from Ecoinvent {GLO} U']

In [30]:
any(
        exc['name'].endswith("S - Copied from Ecoinvent U")
        for exc in ds['exchanges'] 
        if exc.get("functional")
    )

False

In [37]:
registry = rd.Registry()

In [38]:
dp = rn.Datapackage(
    name="agribalyse-3.1.1-delete-aggregated-ecoinvent",
    description="Delete ecoinvent activities from Agribalyse 3.1.1 given as aggregated processes. You need to use the `Process identifier` value, the names are not unique and include modified activities based on ecoinvent which shouldn't be deleted.",
    contributors=[
        {"title": "Chris Mutel", "path": "https://chris.mutel.org/", "role": "author"},
    ],
    mapping_target=rn.MappingConstants.SIMAPRO_CSV,
    mapping_source=rn.MappingConstants.SIMAPRO_CSV,
    version="2.1.0",
    source_id="agribalyse-3.1.1",
    target_id="agribalyse-3.1.1",
    graph_context=["nodes"],
    homepage="https://www.data.gouv.fr/fr/datasets/agribalyse-3-1-1-synthese/",
)
dp.add_data("delete", [{'source': {'identifier': x, 'name': y}} for x, y in data])
registry.add_file(dp.to_json(Path("agribalyse-3.1.1-delete-aggregated-ecoinvent.json")), replace=True)

PosixPath('/Users/cmutel/Code/randonneur_data/randonneur_data/data/agribalyse-3.1.1-delete-aggregated-ecoinvent.xz')

In [51]:
names = {
    exc['name']
    for ds in sp.data 
    for exc in ds.get('exchanges') 
    if exc['name'].endswith(", S - Copied from Ecoinvent U")
}

In [52]:
data = [
    {"source": {"name": name}, "target": {"name": name.replace(", S - Copied from Ecoinvent U", ", U")}} 
    for name in names
]

In [53]:
for ds in data:
    if ds['source']['name'] == 'Sawnwood, board, softwood, raw, dried (u=20%) {GLO}| market for | Cut-off, S - Copied from Ecoinvent U':
        ds['target'] = {
            'name': 'Sawnwood, board, softwood, raw, dried (u=20%) {RoW}| market for sawnwood, board, softwood, raw, dried (u=20%) | Cut-off, U',
        }
        ds['comment'] = 'Wood only used for honey-making in the USA'

In [54]:
for ds in data:
    if 'Electricity, low voltage {RoW}| market for | Cut-off, S - Copied from Ecoinvent U' in ds['source']['name']:
        ds['target']['name'] = 'Electricity, low voltage {GLO}| market group for | Cut-off, U'
        ds['comment'] = 'Manual match as RoW market not in 3.8'

In [55]:
for ds in data:
    if 'Transmission network, long-distance {UCTE}| construction | Cut-off, S - Copied from Ecoinvent U' in ds['source']['name']:
        ds['target']['name'] = 'Transmission network, electricity, high voltage {GLO}| market for | Cut-off, U'
        ds['comment'] = "The long-distance option was deleted in ecoinvent 3.9.1 without any given upgrade path. Substitute with related; this is only used in Martinique and Guadeloupe."

In [20]:
sp_migration = registry.get_file("simapro-ecoinvent-3.8-cutoff")

In [35]:
for ds in sp_migration['replace']:
    if 'Transmission network, long-distance' in ds['source']['name']: # and 'market group' in ds['source']['name'] and '{GLO}' in ds['source']['name']:
        print(ds)

{'source': {'identifier': 'EI3ARUNI000011519608846', 'name': 'Transmission network, long-distance {UCTE}| construction | Cut-off, U', 'platform_id': 'F2FAD5BA-3B9B-41A2-ADAB-A7DC80CBCEF6'}, 'target': {'filename': '38046f77-f0bf-459f-b69e-4ac250673714_c975834b-76cd-431c-a458-98ff9fecb9e5.spold', 'name': 'transmission network construction, long-distance', 'location': 'UCTE', 'reference product': 'transmission network, long-distance', 'unit': 'km'}}
{'source': {'identifier': 'EI3ARUNI000011519608412', 'name': 'Transmission network, long-distance {RoW}| construction | Cut-off, U', 'platform_id': '2298F915-09D3-4AFA-9C9E-F4ABE3A7D884'}, 'target': {'filename': 'c266d4cd-d612-45b0-b7ab-2c68a5293468_c975834b-76cd-431c-a458-98ff9fecb9e5.spold', 'name': 'transmission network construction, long-distance', 'location': 'RoW', 'reference product': 'transmission network, long-distance', 'unit': 'km'}}
{'source': {'identifier': 'EI3ARUNI000011519608529', 'name': 'Transmission network, long-distance {G

In [56]:
dp = rn.Datapackage(
    name="agribalyse-3.1.1-restore-simapro-ecoinvent-names",
    description="""Restore names of linked ecoinvent processes to original SimaPro form.
Changes `Foo, S - Copied from Ecoinvent U` to `Foo, U`, i.e. the standard way that ecoinvent processes are given in SimaPro.
Done in preparation for harmonization and linking via standard Randonneur transformations like `simapro-ecoinvent-3.8-cutoff`.""",
    contributors=[
        {"title": "Chris Mutel", "path": "https://chris.mutel.org/", "role": "author"},
    ],
    mapping_target=rn.MappingConstants.SIMAPRO_CSV,
    mapping_source=rn.MappingConstants.SIMAPRO_CSV,
    version="1.1.0",
    source_id="agribalyse-3.1.1",
    target_id="agribalyse-3.1.1",
    graph_context=["edges"],
    homepage="https://www.data.gouv.fr/fr/datasets/agribalyse-3-1-1-synthese/",
)
dp.add_data("replace", data)
registry.add_file(dp.to_json(Path("agribalyse-3.1.1-restore-simapro-ecoinvent-names.json")), replace=True)

PosixPath('/Users/cmutel/Code/randonneur_data/randonneur_data/data/agribalyse-3.1.1-restore-simapro-ecoinvent-names.xz')

In [22]:
r_data = registry.get_file('agribalyse-3.1.1-restore-simapro-ecoinvent-names')['replace']

print([
    o 
    for o in r_data
    if o['source']['name'] == 'Barley grain, feed {CA-QC}| production | Cut-off, S - Copied from Ecoinvent U'
])

r_data = None

[]


In [28]:
sp.randonneur("agribalyse-3.1.1-restore-simapro-ecoinvent-names")
sp.randonneur("simapro-ecoinvent-3.8-cutoff", fields=["name"])

2024-09-11 21:42:21.786 | INFO     | randonneur.edges:migrate_edges_with_stored_data:71 - Loaded transformation data simapro-ecoinvent-3.8-cutoff from registry with following verbs: ['replace']
2024-09-11 21:42:21.787 | INFO     | randonneur.generic_transformation:generic_transformation:30 - Can apply the following transformation verbs: ['replace']


In [16]:
sp.apply_strategies()

Applying strategy: set_metadata_using_single_functional_exchange
Applying strategy: override_process_name_using_single_functional_exchange
Applying strategy: drop_unspecified_subcategories
Applying strategy: split_simapro_name_geo
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: link_iterable_by_fields
Applying strategy: match_internal_simapro_simapro_with_unit_conversion
Applied 7 strategies in 10.20 seconds


In [33]:
sp.match_database("ecoinvent-3.8-cutoff")

Applying strategy: link_iterable_by_fields


In [34]:
sp.statistics()

16807 datasets, including 487 multifunctional datasets
	357539 exchanges
	Links to the following databases:
		ecoinvent-3.8-cutoff (81697 exchanges)
		agrobalyse-3.1.1 (68532 exchanges)
	207310 unlinked exchanges (3 types)
		Type biosphere: 2168 unique unlinked exchanges
		Type production: 282 unique unlinked exchanges
		Type technosphere: 542 unique unlinked exchanges


(16807, 357539, 207310, 487)

In [41]:
for exc in sp.unlinked:
    if exc['type'] == 'production':
        break

In [42]:
exc

{'name': 'Bagging, storage and loading of unpelleted by-products, grinded by-products, industrial production, French production mix, at plant, 1 kg unpelleted by-products at the exit gate (POUi) {FR} U',
 'unit': 'kg',
 'waste_type': 'not defined',
 'category': 'Agricultural',
 'comment': '(2,3,2,2,2);',
 'line_no': 283395,
 'amount': 1.0,
 'allocation': 100.0,
 'type': 'production',
 'functional': True,
 'properties': {'manual_allocation': 100.0}}

In [43]:
for ds in sp.data:
    if exc in ds['exchanges']:
        break

In [44]:
ds

{'simapro_project': 'AGRIBALYSE - Unit',
 'exchanges': [{'name': 'Paper bag, 30 kg capacity, at plant {RER} U',
   'unit': 'kg',
   'comment': '(2,3,2,2,2);weighted average from 13 plants',
   'line_no': 283402,
   'amount': 0.00247,
   'uncertainty type': 0,
   'loc': 0.00247,
   'type': 'technosphere',
   'simapro_category': 'Materials/fuels',
   'input': ('agrobalyse-3.1.1', 'AGRIBALU000000003110792')},
  {'name': 'market for electricity, medium voltage',
   'unit': 'kWh',
   'comment': '(2,3,2,2,2),',
   'line_no': 283405,
   'amount': 0.08388000000000001,
   'uncertainty type': 0,
   'loc': 0.0233,
   'original unit before conversion': 'kWh',
   'unit conversion factor': 3.6,
   'type': 'technosphere',
   'simapro_category': 'Electricity/heat',
   'filename': '953e308d-ea8d-44cf-ba17-f762486ad89a_759b89bd-3aa6-42ad-b767-5bb9ef5d331d.spold',
   'location': 'FR',
   'reference product': 'electricity, medium voltage',
   'input': ('ecoinvent-3.8-cutoff', 'a3fd348c08e62cc9daf378cbc73e

In [39]:
for ds in sp.data:
    func = [exc for exc in ds['exchanges'] if exc.get('functional')]
    if len(func) == 1 and func[0]['name'] != ds['name']:
        break

In [40]:
ds

{'simapro_project': 'AGRIBALYSE - Unit',
 'exchanges': [{'name': 'Average compost, from green waste, biowaste, sludge, manure, slurry (amendment) {RER} U',
   'unit': 'kg',
   'comment': 'Organic fertilizer',
   'line_no': 6937817,
   'amount': 4000.0,
   'uncertainty type': 2,
   'scale': 0.06538253493977626,
   'loc': 8.294049640102028,
   'negative': False,
   'original unit before conversion': 'ton',
   'unit conversion factor': 1000.0,
   'type': 'technosphere',
   'simapro_category': 'Materials/fuels',
   'input': ('agrobalyse-3.1.1', 'AGRIBALU000000003100742')},
  {'name': 'Fertilizing, with spreader/broadcaster, 500 l (orchard) {FR} U',
   'unit': 's',
   'comment': 'Fertilizing, with spreader/broadcaster, 500 L (greenhouse) - manuring',
   'line_no': 6937818,
   'amount': 10800.0,
   'uncertainty type': 2,
   'scale': 0.06538253493977626,
   'loc': 9.287301413112312,
   'negative': False,
   'original unit before conversion': 'hr',
   'unit conversion factor': 3600.0,
   'type

In [37]:
for ds in sp.data:
    if ds['name'] == 'Propane, burned in industrial furnace, at plant {FR} U':
        print(ds['unit'])

In [26]:
md = registry.get_file("simapro-ecoinvent-3.8-cutoff")

In [27]:
for dct in md['replace']:
    if dct['source']['name'].startswith("Sodium chloride, powder"):
        print(dct)

{'source': {'identifier': 'EI3ARUNI000011519605303', 'name': 'Sodium chloride, powder {RoW}| production | Cut-off, U', 'platform_id': '48F01B13-F2EA-4AA7-99FF-9561EC25843A'}, 'target': {'filename': '0f28e9fc-ef10-4dc7-afda-8d2ec3b7e1bf_edcfa83a-363a-4b2d-8d6a-612e32dfcdfc.spold', 'name': 'sodium chloride production, powder', 'location': 'RoW', 'reference product': 'sodium chloride, powder', 'unit': 'kg'}}
{'source': {'identifier': 'EI3ARUNI000011519605129', 'name': 'Sodium chloride, powder {GLO}| market for | Cut-off, U', 'platform_id': 'FF9A60C8-831A-498E-8991-F4452B905508'}, 'target': {'filename': '40223e49-e3a8-4da6-ac87-3b9068efb227_edcfa83a-363a-4b2d-8d6a-612e32dfcdfc.spold', 'name': 'market for sodium chloride, powder', 'location': 'GLO', 'reference product': 'sodium chloride, powder', 'unit': 'kg'}}
{'source': {'identifier': 'EI3ARUNI000011519621101', 'name': 'Sodium chloride, powder {CN-SC}| rare earth oxides production, from rare earth oxide concentrate, 70% REO | Cut-off, U',

In [ ]:
print()

In [39]:
print(registry)

`randonneur_data` with 21 files:
	agribalyse-3.1.1-delete-aggregated-ecoinvent
		Description: Delete ecoinvent activities from Agribalyse 3.1.1 given as aggregated processes
		Source: None
		Target: agribalyse-3.1.1
		Graph objects to be modified: ['nodes']
		Authors: Chris Mutel (author)
		Version: 1.0.0
		Licenses: ['CC-BY-4.0']
	ecoinvent-2.2-biosphere-context-ecoinvent-3.0-biosphere-context
		Description: Convert context (category and subcategory) labels from ecoinvent 2 to 3 standards
		Source: ecoinvent-2.2-biosphere
		Target: ecoinvent-3.0-biosphere
		Graph objects to be modified: ['nodes', 'edges']
		Authors: Chris Mutel (author)
		Version: 1.0.0
		Licenses: ['CC BY 4.0']
	ecoinvent-2.2-biosphere-ecoinvent-3.0-biosphere
		Description: Convert biosphere flow names from ecoinvent 2 to 3
		Source: ecoinvent-2.2-biosphere
		Target: ecoinvent-3.0-biosphere
		Graph objects to be modified: ['nodes', 'edges']
		Authors: Chris Mutel (author)
		Version: 1.0.0
		Licenses: ['CC BY 4.0']
	e